In [1]:
from pyspark.sql import SparkSession

spark_session = SparkSession.builder\
        .master("spark://192.168.2.97:7077") \
        .appName("Q3")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.cores.max", 2)\
        .getOrCreate()

spark_context = spark_session.sparkContext
        #.config("spark.jars.packages", "uk.co.gresearch.spark:spark-extension_2.12:2.4.0-3.3")\
spark_context.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/03/13 13:52:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
redComm1 = spark_session.read.json("hdfs://192.168.2.97:50000/user/ubuntu/RC_2008-03.json")
redComm2 = spark_session.read.json("hdfs://192.168.2.97:50000/user/ubuntu/RC_2008-04.json")

In [3]:
def json_file_read_and_count(json_filename):
    df = spark_session.read.json(json_filename)

    my_df_subreddit = df.select("subreddit")\
        .groupBy("subreddit")\
        .count()\
        .orderBy("count", ascending=False)
    
    return my_df_subreddit

In [4]:
redComm1.printSchema()

root
 |-- body: string (nullable = true)
 |-- subreddit: string (nullable = true)



In [5]:
subs = redComm1.select('subreddit')
subs.take(5)

[Row(subreddit='reddit.com'),
 Row(subreddit='reddit.com'),
 Row(subreddit='reddit.com'),
 Row(subreddit='reddit.com'),
 Row(subreddit='reddit.com')]

In [6]:
subCountM1 = subs\
    .groupBy("subreddit")\
    .count()\
    .orderBy("count", ascending=False)

In [7]:
subCountM2 = redComm2\
    .select('subreddit')\
    .groupBy('subreddit')\
    .count()\
    .orderBy('count', ascending=False)

In [8]:
subCountM1.show()

+-------------+------+
|    subreddit| count|
+-------------+------+
|   reddit.com|207092|
|     politics| 83375|
|  programming| 34642|
|         pics| 25352|
|      science| 22874|
|     business| 14555|
|    worldnews| 12725|
|        funny| 10223|
|entertainment|  9403|
|       gaming|  9313|
|         nsfw|  4036|
|          WTF|  3276|
|      offbeat|  2709|
|   technology|  2645|
|    Economics|  2313|
|        obama|  1863|
|      gadgets|  1714|
|  environment|  1323|
|    AskReddit|  1133|
|       comics|  1029|
+-------------+------+
only showing top 20 rows



In [9]:

from pyspark.sql.functions import col
joinedCount = subCountM2.select(col("subreddit").alias("subreddit_new"), col("count").alias("count_new"))\
    .join(subCountM1.select(col("subreddit").alias("subreddit_old"), col("count").alias("count_old")), col("subreddit_new") == col("subreddit_old"),"left")

In [10]:
joinedCount.show()

+-------------+---------+-------------+---------+
|subreddit_new|count_new|subreddit_old|count_old|
+-------------+---------+-------------+---------+
|   reddit.com|   140944|   reddit.com|   207092|
|     politics|    64532|     politics|    83375|
|  programming|    40958|  programming|    34642|
|         pics|    34426|         pics|    25352|
|      science|    31727|      science|    22874|
|    worldnews|    28143|    worldnews|    12725|
|     business|    25699|     business|    14555|
|        funny|    20824|        funny|    10223|
|entertainment|    19519|entertainment|     9403|
|       gaming|    10156|       gaming|     9313|
|   technology|     9460|   technology|     2645|
|    AskReddit|     4738|    AskReddit|     1133|
|          WTF|     4413|          WTF|     3276|
|      offbeat|     3626|      offbeat|     2709|
|      gadgets|     2253|      gadgets|     1714|
|         nsfw|     2114|         nsfw|     4036|
|    Economics|     1982|    Economics|     2313|


In [11]:
new_df = joinedCount.withColumn('count_diff', joinedCount.count_new - joinedCount.count_old)
new_df.show()

+-------------+---------+-------------+---------+----------+
|subreddit_new|count_new|subreddit_old|count_old|count_diff|
+-------------+---------+-------------+---------+----------+
|   reddit.com|   140944|   reddit.com|   207092|    -66148|
|     politics|    64532|     politics|    83375|    -18843|
|  programming|    40958|  programming|    34642|      6316|
|         pics|    34426|         pics|    25352|      9074|
|      science|    31727|      science|    22874|      8853|
|    worldnews|    28143|    worldnews|    12725|     15418|
|     business|    25699|     business|    14555|     11144|
|        funny|    20824|        funny|    10223|     10601|
|entertainment|    19519|entertainment|     9403|     10116|
|       gaming|    10156|       gaming|     9313|       843|
|   technology|     9460|   technology|     2645|      6815|
|    AskReddit|     4738|    AskReddit|     1133|      3605|
|          WTF|     4413|          WTF|     3276|      1137|
|      offbeat|     3626

In [18]:
from pyspark.sql.functions import trim, regexp_replace, lower, explode, trim, split
from pyspark.sql.types import StructType, StructField, StringType

years = ["08"]
months = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]

empty_schema = StructType([StructField("subreddit", StringType(), True),])
df_all_years = spark_session.createDataFrame([], empty_schema)


for year in years:
    i = 1
    while i < 5:
        lastMonth = json_file_read_and_count("hdfs://192.168.2.97:50000/user/ubuntu/RC_2008-"+ months[i-1] + ".json")
        currentMonth = json_file_read_and_count("hdfs://192.168.2.97:50000/user/ubuntu/RC_2008-"+ months[i] + ".json")
        
        joinedCount = currentMonth.select(col("subreddit").alias("subreddit_new"), col("count").alias("count_new"))\
            .join(lastMonth.select(col("subreddit").alias("subreddit_old"), col("count").alias("count_old")), col("subreddit_new") == col("subreddit_old"),"left")
        new_df = joinedCount.withColumn('count_diff', joinedCount.count_new - joinedCount.count_old)
        
        if i >= 2:
            prevSubname = 'subreddit_'+str(months[i-1])
        else:
            prevSubname = 'subreddit'
        
        subname = 'subreddit_'+str(months[i])
        countname = 'count_diff_'+str(months[i])
        
        df_all_years = df_all_years.join(new_df.select(col("subreddit_new").alias(subname), col("count_diff").alias(countname)), col(prevSubname) == col(subname),"right")
        i += 1
        
df_all_years.drop(col("subreddit")).show()

+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+
| subreddit_02|count_diff_02| subreddit_03|count_diff_03| subreddit_04|count_diff_04| subreddit_05|count_diff_05|
+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+
|    AskReddit|          715|    AskReddit|          274|    AskReddit|         3605|    AskReddit|         1416|
|    Economics|          397|    Economics|         1902|    Economics|         -331|    Economics|          266|
|          WTF|          762|          WTF|         2496|          WTF|         1137|          WTF|         1909|
|      atheism|           89|      atheism|          822|      atheism|          938|      atheism|         1863|
|     business|          -39|     business|        10939|     business|        11144|     business|         -665|
|       comics|          503|       comics|          485|       comics|          487|   

In [ ]:
spark_session.stop()